# 타이타닉 데이터 분석을 위한 Spark SQL 응용

### 개요

* 이 실습에서는 타이타닉 데이터셋을 사용하여 Spark SQL을 활용해 이전과 다른 새로운 분석을 수행합니다. 
* 분석 목표는 운임(Fare) 분포, 항구별 승객 분포, 형제자매/배우자 수에 따른 생존율 등을 분석하는 것입니다.


### 1. 환경 설정 및 데이터 불러오기

#### SparkSession 및 데이터 로드

#### Colab에서 환경 설정

In [ ]:
#!sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# findspark를 사용해 Spark 초기화
import findspark
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

In [1]:
# SparkSession을 설정합니다.
from pyspark.sql import SparkSession

# SparkSession 생성
# 'Titanic Data Analysis'이라는 애플리케이션 이름으로 Spark 세션을 초기화합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = SparkSession.builder.appName("Titanic Data Analysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/07 13:17:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/07 13:17:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/07 13:17:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/07 13:17:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### TODO: 이전까지의 내용을 기반으로 주석에 쓰여있는 결과를 낼 수 있도록 코드를 채워주세요. SQL이나 Spark SQL을 사용해서 결과를 잘 만들면 됩니다.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

# SparkSession 설정
# 'Titanic Advanced Analysis'라는 이름의 Spark 세션을 생성합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.html
spark = SparkSession.builder.appName("Titanic Advanced Analysis").getOrCreate()

# 스키마 정의
# 데이터 스키마를 명시적으로 정의하여 각 열의 데이터 타입을 지정합니다.
# 이 스키마는 CSV 파일을 읽을 때 적용됩니다.
schema = ___________([
    ___________("PassengerId", ___________(), True),   # 승객 ID (int)
    ___________("Survived", ___________(), True),      # 생존 여부 (0 = 사망, 1 = 생존) (int)
    ___________("Pclass", ___________(), True),        # 승객 등급 (1, 2, 3) (int)
    ___________("Name", ___________(), True),           # 승객 이름 (String)
    ___________("Sex", ___________(), True),            # 성별 (String)
    ___________("Age", ___________(), True),             # 나이 (float)
    ___________("SibSp", ___________(), True),         # 동반한 형제/배우자 수 (int)
    ___________("Parch", ___________(), True),         # 동반한 부모/자녀 수 (int)
    ___________("Ticket", ___________(), True),         # 티켓 번호 (String)
    ___________("Fare", ___________(), True),            # 운임 요금  (float)
    ___________("Cabin", ___________(), True),          # 객실 번호 (String)
    ___________("Embarked", ___________(), True)        # 탑승 항구 (C = Cherbourg, Q = Queenstown, S = Southampton) (String)
])

# CSV 파일 읽기
# 정의된 스키마와 옵션을 사용하여 CSV 파일을 읽어 DataFrame으로 로드합니다.
# header 옵션은 첫 줄을 헤더로 사용할지 여부를 설정하며, delimiter 옵션은 필드 구분자를 지정합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html
df = spark.read.schema(schema).format("csv")\
    .option("header", "true")\
    .option("delimiter", ",")\
    .load("../data/titanic.csv")

# 데이터 출력
# show() 메서드를 사용하여 상위 5개 행을 출력하여 데이터를 확인합니다.
df.show(5)


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### 2. 운임(Fare) 분석


#### 운임 분포 확인

##### 설명
* `selectExpr()`: Spark DataFrame에서 SQL 표현식을 사용하여 데이터를 선택할 수 있게 해주는 메서드입니다. SQL과 유사한 문법을 사용해 집계 연산, 데이터 변환, 또는 열 선택을 간단하게 수행할 수 있습니다.
* `MIN(Fare) AS MinFare`: Fare 열에서 최소값을 계산하고, 결과 열의 이름을 MinFare로 지정합니다.
* `MAX(Fare) AS MaxFare`: Fare 열에서 최대값을 계산하고, 결과 열의 이름을 MaxFare로 지정합니다.
* `AVG(Fare) AS AvgFare`: Fare 열의 평균을 계산하고, 결과 열의 이름을 AvgFare로 지정합니다.
* `STDDEV(Fare) AS StdDevFare`: Fare 열의 표준편차를 계산하고, 결과 열의 이름을 StdDevFare로 지정합니다.

In [ ]:
# 운임의 최소, 최대, 평균, 표준편차 계산

+-------+--------+-----------------+-----------------+
|MinFare| MaxFare|          AvgFare|       StdDevFare|
+-------+--------+-----------------+-----------------+
|    0.0|512.3292|32.20420804114722|49.69342916316158|
+-------+--------+-----------------+-----------------+



#### 운임 범위에 따른 승객 수 분포

##### 설명
* `withColumn()`: 새로운 열 FareRange를 생성합니다.
* `when()`: 조건에 따라 Fare 값을 구간별로 나누고, 각 구간에 해당하는 문자열 값을 할당합니다.
* `F.when()`: 조건이 참일 경우 값을 반환합니다.
* `F.otherwise()`: 위의 모든 조건이 거짓일 경우 기본값을 반환합니다.
* `groupBy()`: FareRange 열을 기준으로 그룹화합니다.
* `count()`: 각 구간에 해당하는 승객 수를 계산합니다.
* `orderBy()`: FareRange를 기준으로 결과를 정렬합니다.


In [ ]:
from pyspark.sql import functions as F

# 운임(Fare)을 구간별로 나누어 새로운 열 FareRange를 생성합니다.

# 구간별 승객 수를 계산하고 정렬합니다.


+---------+-----+
|FareRange|count|
+---------+-----+
|     0-10|  336|
|    10-50|  394|
|     100+|   53|
|   50-100|  108|
+---------+-----+



### 3. 항구별 승객 분포 분석

#### 항구별 승객 수 집계

##### 설명

* `groupBy()`: Embarked 열을 기준으로 승객을 그룹화합니다. 각 항구에서 승선한 승객들을 그룹으로 묶습니다.
* `count()`: 각 그룹(항구) 내의 승객 수를 계산합니다.
* `orderBy()`: count 열을 기준으로 내림차순(큰 값부터 작은 값 순서로) 정렬하여 승객 수가 많은 항구부터 보여줍니다.


In [ ]:
# 항구(Embarked)별 승객 수 계산


+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    null|    2|
+--------+-----+



#### 항구별 평균 운임 계산

##### 설명
* `groupBy()`: Embarked 열을 기준으로 승객을 그룹화합니다. 각 항구에서 승선한 승객들을 그룹으로 묶습니다.
* `agg()`: 집계 연산을 수행합니다. 여기서는 avg() 함수를 사용해 운임(Fare)의 평균값을 계산합니다.
* `F.avg()`: Fare 열의 평균값을 계산하는 함수입니다.
* `.alias()`: 계산된 평균값에 별칭(AvgFare)을 부여하여 결과 열의 이름을 지정합니다.
* `orderBy()`: AvgFare 열을 기준으로 내림차순으로 정렬하여 평균 운임이 높은 항구부터 보여줍니다.

In [ ]:
# 항구별 평균 운임


+--------+------------------+
|Embarked|           AvgFare|
+--------+------------------+
|    null|              80.0|
|       C|  59.9541444891975|
|       S|27.079811787012943|
|       Q|13.276029865463059|
+--------+------------------+



### 4. 데이터 저장

#### 분석 결과를 CSV 파일로 저장

* `DataFrame.write.csv()` 메서드를 사용할 때, 결과는 단일 CSV 파일이 아닌 폴더로 저장
##### CSV 파일
- 기본적으로 `csv` 파일은 여러 파티션으로 저장되기 때문에 `part-00000`, `part-00001` 등의 파일들이 생성됩니다.
- 각 `part-` 파일에는 분산된 데이터의 일부가 저장됩니다. (예: `part-00000.csv`, `part-00001.csv` 등)

##### _SUCCESS 파일
- 작업이 성공적으로 완료되었음을 나타내는 빈 파일이 `_SUCCESS`라는 이름으로 생성됩니다.
- 이 파일은 데이터 저장이 정상적으로 완료되었음을 표시합니다.


* `coalesce()`를 사용하지 않으면 데이터가 여러 CSV 파일로 나뉘어 저장되며, 각 파일은 Spark의 분산 처리 결과를 나타냅니다. 대규모 데이터 처리에서는 이러한 다중 파일 저장이 일반적이며, 여러 노드에서 데이터를 병렬로 처리할 때 효율적

In [10]:
# 폴더로
df.write.csv("../data/titanic_advanced_analysis", header=True)

In [13]:
# 데이터를 하나의 파티션으로 병합한 후 CSV 파일로 저장합니다.
# 여기서는 규모가 크지 않아 차이가 없습니다.
df.coalesce(1).write.csv("../data/titanic_advanced_analysis_coalesce", header=True)

##### 설명
* `.write.csv()`: 분석 결과를 CSV 파일로 저장합니다. `header=True`로 설정하여 열 이름을 포함합니다.


### 5. SparkSession 중지

In [ ]:
spark.stop()  # Spark 세션 중지